# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Calculation started (calculation_id=84c95ac5-1630-b7e5-c200-564f21d85ae3) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=e0c95ac5-1bb5-ecb3-93a5-a8cab479b207) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=d2c95ac5-20b9-f8bf-85c4-effa68cea39b) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'amehroke3'

Calculation started (calculation_id=0cc95ac5-21ea-dca4-3546-f4edd875d94a) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    trips = trips.filter(trips["trip_distance"] >= 2)
    return trips

Calculation started (calculation_id=d4c95ac5-234b-569b-de9a-657eb1c6772b) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.3 [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [16]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    manhattan_lookup = lookup.filter(lookup.Borough == "Manhattan").select("LocationID")
    
    manhattan_trips = trips.join(manhattan_lookup, trips.DOLocationID == manhattan_lookup.LocationID)
    
    manhattan_trips = manhattan_trips.withColumn("passenger_count", col("passenger_count").cast("int"))

    grouped_trips = manhattan_trips.groupBy("DOLocationID") \
        .sum("passenger_count") \
        .withColumnRenamed("sum(passenger_count)", "pcount") \
        .orderBy("pcount", ascending=False)
    
    mtrips = grouped_trips.limit(20)
    
    return mtrips

Calculation started (calculation_id=0ac95ac7-8eca-639e-eaa8-7a0f214d8d3e) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.4 [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [29]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    top_20_ids = [row['DOLocationID'] for row in mtrips.collect()]
    
    total_trips = trips.groupBy("PULocationID") \
        .agg({'total_amount': 'avg', '*': 'count'}) \
        .withColumnRenamed("avg(total_amount)", "avg_total_amount") \
        .withColumnRenamed("count(1)", "total_trip_count")
    
    trips_to_top_20 = trips.filter(trips['DOLocationID'].isin(top_20_ids))
    top_20_trips = trips_to_top_20.groupBy("PULocationID") \
        .count() \
        .withColumnRenamed("count", "top_20_trip_count")
    
    result = total_trips.join(top_20_trips, on="PULocationID", how="inner")
    
    result = result.withColumn(
        "weighted_profit", 
        (result['avg_total_amount'] * (result['top_20_trip_count'] / result['total_trip_count']))
    )
    
    wp = result.select("PULocationID", "weighted_profit").orderBy("weighted_profit", ascending=False)
    
    return wp



Calculation started (calculation_id=52c95adb-703f-8d58-8361-6402e28609c8) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.5 [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [8]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    final = wp.join(lookup, wp["PULocationID"] == lookup["LocationID"], "left") \
               .select("Zone", "Borough", "weighted_profit") \
               .orderBy("weighted_profit", ascending=False) \
               .limit(20)
    return final

Calculation started (calculation_id=3cc95ac5-2ee0-f256-fc4d-585f38bc2120) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [9]:
trips,lookup = load_data(size='small')

Calculation started (calculation_id=56c95ac5-3008-4dde-6948-7e6f33e7377d) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Trip Count:  7667792



In [10]:
mtrips = manhattan_trips(trips,lookup)

Calculation started (calculation_id=8cc95ac5-6992-1136-da21-3eca0d89bec8) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [11]:
wp = weighted_profit(trips, mtrips)

Calculation started (calculation_id=a4c95ac5-6eda-fbe3-e425-a26e5ec06275) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [12]:
wp.show()

Calculation started (calculation_id=3ac95ac5-901e-5988-9962-038a3bcbe969) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+------------------+
|PULocationID|   weighted_profit|
+------------+------------------+
|          10|21.060544583116087|
|         194|20.434121515335203|
|         215|19.806353214571025|
|          93|19.525570123421428|
|           2|           18.5028|
|         138|18.175503156233436|
|           5|           17.6256|
|         132| 17.04797310829457|
|         206|  16.2431660899654|
|          24|14.711553041789351|
|         171|14.345707821376665|
|         156|   13.861982421875|
|         130| 12.42753436069335|
|         199|           12.3675|
|         157|11.908022338501244|
|           8|11.652853535353538|
|         219|11.561567175193563|
|         105|10.202130177514793|
|         237|10.046279042326216|
|          56| 9.913064412510566|
+------------+------------------+
only showing top 20 rows



In [13]:
final = final_output(wp, lookup)

Calculation started (calculation_id=4ec95ac5-a952-9108-1b59-011f10ee9636) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [14]:
final.show()

Calculation started (calculation_id=08c95ac5-ae75-37b8-c3f9-8580db623e4f) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|  21.0605445831161|
|     Randalls Island|    Manhattan|20.434121515335214|
|       South Jamaica|       Queens|19.806353214570994|
|Flushing Meadows-...|       Queens|19.525570123421403|
|         Jamaica Bay|       Queens|           18.5028|
|   LaGuardia Airport|       Queens|18.175503156235756|
|       Arden Heights|Staten Island|           17.6256|
|         JFK Airport|       Queens| 17.04797310829401|
|Saint George/New ...|Staten Island|  16.2431660899654|
|        Bloomingdale|    Manhattan|14.711553041789475|
|  Murray Hill-Queens|       Queens|14.345707821376658|
|     Mariners Harbor|Staten Island|13.861982421874998|
|             Jamaica|       Queens| 12.42753436069332|
|       Rikers Island|        Bronx|           12.3675|
|             Maspeth|   

In [30]:
main('large', 's3://cse6242-amehroke3/output-large3')

Calculation started (calculation_id=a2c95adb-b7b2-84bd-a3f8-02dc0a7482f0) in (session=cec95ac4-dea9-c2ed-afc8-526ffcaa239b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
amehroke3
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens| 29.36045577913084|
|Flushing Meadows-...|       Queens|27.304845733617682|
|       South Jamaica|       Queens|26.294916239873473|
|     Randalls Island|    Manhattan| 24.15098994022753|
|        Astoria Park|       Queens|21.706417112147527|
|Briarwood/Jamaica...|       Queens|19.945064631789332|
|Springfield Garde...|       Queens|19.468309288781906|
|             Jamaica|       Queens|19.283943000137903|
|              Corona|       Queens|18.228769248155967|
|   LaGuardia Airport|       Queens|   18.181338808373|
|         Jamaica Bay|       Queens|17.100529446757896|
|             Maspeth|       Queens| 17.00545064007954|
|Eltingville/Annad...|Staten Island| 16.83776475694445|
|         JFK Airport|       Queens|16.777725348